In [6]:
# 导入各种包
import paddle
import paddle.vision.models as models
from paddle.static import InputSpec as Input
from paddle.vision.datasets import Cifar10
import paddle.vision.transforms as T
from paddleslim.dygraph import L1NormFilterPruner

In [7]:
# 网络定义和数据集加载 
# 使用mobilenet_V1的模型
net = models.mobilenet_v1()
inputs = Input(shape=[None, 3, 32, 32], dtype='float32', name='image')
labels = Input(shape=[None, 1], dtype='int64', name='label')
optimizer = paddle.optimizer.Momentum(learning_rate=0.1, parameters=net.parameters())
model = paddle.Model(net, inputs, labels)
model.prepare(
    optimizer=optimizer,
    loss=paddle.nn.CrossEntropyLoss(),
    metrics=paddle.metric.Accuracy(topk=(1, 5))
)

transforms = T.Compose([
    T.Transpose(),
    T.Normalize([127.5], [127.5])
])

train_dataset = Cifar10(mode='train', transform=transforms)
test_dataset = Cifar10(mode='train', transform=transforms)


item    59/41626 [..............................] - ETA: 52s - 1ms/item

Cache file /home/baiduiov/.cache/paddle/dataset/cifar/cifar-10-python.tar.gz not found, downloading https://dataset.bj.bcebos.com/cifar/cifar-10-python.tar.gz 
Begin to download


item  2941/41626 [=>............................] - ETA: 12s - 317us/item

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



item 15015/41626 [=========>....................] - ETA: 6s - 260us/item

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



item 26985/41626 [==================>...........] - ETA: 3s - 249us/item

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



item 38906/41626 [===========================>..] - ETA: 0s - 247us/item

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:

# 训练模型
model.fit(train_dataset, epochs=2, batch_size=128, verbose=1)



The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2


/home/baiduiov/tools/miniconda3/lib/python3.9/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.


step 391/391 [==============================] - loss: 1.8013 - acc_top1: 0.3318 - acc_top5: 0.8296 - 1s/step          
Epoch 2/2
step 391/391 [==============================] - loss: 1.4754 - acc_top1: 0.4599 - acc_top5: 0.9125 - 1s/step          


In [9]:
# 计算剪裁之前的模型相关信息。使用paddle.flops函数
flops = paddle.flops(net, input_size=[1, 3, 32, 32], print_detail=True)


<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
<class 'paddle.nn.layer.activation.ReLU'>'s flops has been counted
<class 'paddle.nn.layer.pooling.AdaptiveAvgPool2D'>'s flops has been counted
<class 'paddle.nn.layer.common.Linear'>'s flops has been counted
+-----------------------+-----------------+-----------------+---------+---------+
|       Layer Name      |   Input Shape   |   Output Shape  |  Params |  Flops  |
+-----------------------+-----------------+-----------------+---------+---------+
|        conv2d_0       |  [1, 3, 32, 32] | [1, 32, 16, 16] |   864   |  221184 |
|     batch_norm2d_0    | [1, 32, 16, 16] | [1, 32, 16, 16] |   128   |  16384  |
|        re_lu_0        | [1, 32, 16, 16] | [1, 32, 16, 16] |    0    |    0    |
|        conv2d_1       | [1, 32, 16, 16] | [1, 32, 16, 16] |   288   |  73728  |
|     batch_norm2d_1    | [1, 32, 16, 16] | [1, 32, 16, 16] |   128   |  16384  |
|   

/home/baiduiov/tools/miniconda3/lib/python3.9/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [10]:

# 评估精度 
model.evaluate(test_dataset, batch_size=128, verbose=1)
# 剪裁前的精度为：0.937 
# {'loss': [1.3092904],  'acc': 0.93706}


Eval begin...
step 391/391 [==============================] - loss: 1.3433 - acc_top1: 0.5240 - acc_top5: 0.9352 - 381ms/step          
Eval samples: 50000


{'loss': [1.343312], 'acc_top1': 0.52396, 'acc_top5': 0.9352}

In [ ]:

# 剪裁 
pruner = L1NormFilterPruner(net, [1, 3, 32, 32])
pruner.prune_vars({'conv2d_22.w_0': 0.5, 'conv2d_20.w_0': 0.6}, axis=0)


In [ ]:

# 计算剪裁之后的flops
flops = paddle.flops(net, input_size=[1, 3, 32, 32], print_detail=True)